In [1]:
import os
from datasets import load_dataset
from tqdm import tqdm 

In [2]:
base_dir = "./data_eng_culturax"
output_dir = "./data_eng_culturax_converted"

In [3]:
dataset = load_dataset(base_dir, streaming=True)

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

In [4]:
import pyarrow as pa
import pyarrow.feather as feather 

# 51000000
# 51

content = []
output_file_counter = 51 

for i, row in tqdm(enumerate(dataset["train"])): 
    if i <= 51000000: 
        continue
    content.append(row["text"]) 

    if i % 1e6 == 0 and i != 0: 
        table = pa.Table.from_arrays([pa.array(content)], names=["text"])
        output_filepath = f"./{output_dir}/{str(output_file_counter).zfill(5)}.arrow"

        with pa.CompressedOutputStream(output_filepath, compression="gzip") as stream: 
            writer = pa.RecordBatchStreamWriter(stream, table.schema)
            writer.write_table(table) 
            writer.close()


        output_file_counter += 1
        content = []

57563345it [1:36:11, 9973.00it/s] 


In [6]:
if content: 
    table = pa.Table.from_arrays([pa.array(content)], names=["text"])
    output_filepath = f"./data_vi_culturax_converted/{str(output_file_counter).zfill(5)}.arrow"

    with pa.output_stream(output_filepath) as stream: 
        writer = pa.RecordBatchStreamWriter(stream, table.schema)
        writer.write_table(table) 
        writer.close()

In [8]:
test_dataset = load_dataset("./data_vi_culturax_converted", split="train", streaming=True)

Resolving data files:   0%|          | 0/58 [00:00<?, ?it/s]